In [1]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from random import shuffle
import math
import torch
import pdb
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
import torch.optim as optim
import scipy.interpolate as interp
import collections

In [2]:
def read_data(file_path):
    with h5py.File(file_path, 'r') as f:
    # 查看文件中所有的数据集名称
        dataset_names = list(f.keys())
        print("Datasets in the file:", dataset_names)
        data_dict = {}
        for name in dataset_names:
            if f[name].shape == (): 
                data = f[name][()] 
            else:
                data = f[name][:]
        
            data_dict[name] = data
            
        return data_dict

In [3]:
def save_data(data, file_path):
    with open(file_path , 'wb') as f:
        pickle.dump(data,f)
        f.close()

In [4]:
def save_data_ash5(data, file_path):
    with h5py.File(file_path, 'w') as f:
        keys =list(data.keys())
        for i in range(len(keys)):
            f.create_dataset(keys[i], data=data[keys[i]])

In [5]:
city_names = ["Barcelona","Antwerp", "Bangkok"]
root_path = "D:/ThesisData/processed data/"

In [13]:
city_name = city_names[2]
city_dict = read_data(root_path + city_name + "/input_target/input_target.h5")
print(city_dict['period_template'].shape)

Datasets in the file: ['city_node_num', 'max_vals', 'min_vals', 'period_template', 'target_template', 'trend_template']
(71, 20, 21842, 5)


In [14]:
def percent_divide(city_name, city_dict, indices, root_path, train_ratio, val_ratio, purpose):
    ratio_dict = dict()
    
    keys = list(city_dict.keys())
    train_size = int(train_ratio * city_dict['period_template'].shape[0])
    val_size = int(val_ratio * city_dict['period_template'].shape[0])
    test_size = city_dict['period_template'].shape[0] - train_size - val_size
    city_node_num = city_dict['city_node_num']
    three_element_list = ['period_template', 'target_template', 'trend_template']
    
    for key in keys:
        if key in three_element_list:
            if purpose == "train":
                ratio_dict[key] = city_dict[key][indices][:train_size, :, :city_node_num, :].astype(np.float32)
            elif purpose == "vali":
                ratio_dict[key] = city_dict[key][indices][train_size:train_size+val_size, :, :city_node_num, :].astype(np.float32)
            elif purpose == "test":
                ratio_dict[key] = city_dict[key][indices][train_size+val_size: , :, :city_node_num, :].astype(np.float32)
        elif key == "city_node_num":
            ratio_dict[key] = city_node_num
        else:
            ratio_dict[key] = city_dict[key]
            
    save_data_ash5(ratio_dict, root_path + city_name + f"/input_target/{purpose}.h5")
    print("done")

In [15]:
train_ratio, val_ratio = 0.6, 0.2
purposes = ['train','test','vali']
#Shuffled indicies
indices = np.random.permutation(city_dict['period_template'].shape[0])
for purpose in purposes:
    percent_divide(city_name, city_dict, indices, root_path, train_ratio, val_ratio, purpose)

done
done
done
